# Pydantic: Veri Doğrulama ve Dönüştürme Kütüphanesi

Pydantic, Python'da veri doğrulama, dönüştürme ve serileştirme işlemleri için kullanılan güçlü bir kütüphanedir. 


In [1]:
# !pip install pydantic
from pydantic import BaseModel


# DATA VALIDATION

In [175]:
class User(BaseModel):
    id : int
    name : str
    surname :str
    age :  int

In [176]:
user = User(id=1, name="Enes", surname="KOŞAR", age=22)

In [177]:
user

User(id=1, name='Enes', surname='KOŞAR', age=22)

In [181]:
invalid_user = User(id=1, name="Enes", surname="KOŞAR", age=22)
#invalid_user = User(id=1, name="Enes", surname="KOŞAR", age=0.22)

## DEFAULT DEĞER VERELİM

In [182]:
class User(BaseModel):
    id : int
    name : str = "ENES" 

In [190]:
user = User(id=1)
user

ValidationError: 2 validation errors for User
name
  Field required [type=missing, input_value={'id': 1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
age
  Field required [type=missing, input_value={'id': 1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [191]:
user = User(id=1,name="ENES NAME IS REPLACED")
user

ValidationError: 1 validation error for User
age
  Field required [type=missing, input_value={'id': 1, 'name': 'ENES NAME IS REPLACED'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [192]:
from pydantic import Field

class User(BaseModel):
    id : int
    name : str
    age : int = Field(gt=18, lt=25)

   
user = User(id=1, name="Enes", age=22)
user

User(id=1, name='Enes', age=22)

In [194]:
user = User(id=5, name="Enes", age=22)

## SORU : HANGİ FORMATTA VERİ GELECEĞİNİ ÖNEMSEMİYORSAK ?

In [197]:
from typing import Any

class User(BaseModel):
    id : Any
    name : str
    age : int = Field(gt=18, lt=25)

user = User(id=None, name="Enes", age=22)
user

User(id=None, name='Enes', age=22)

In [13]:
class IncomingResponse(BaseModel):
    api_response  : Any
    
api_response = IncomingResponse(api_response="hi")
api_response = IncomingResponse(api_response=123)
api_response = IncomingResponse(api_response=[1,2,3,4,5])
api_response = IncomingResponse(api_response={"a":1,"b":2})
api_response = IncomingResponse(api_response=True)
api_response = IncomingResponse(api_response=False)
api_response = IncomingResponse(api_response=None)
api_response = IncomingResponse(api_response={"id":1,"name":"Enes","age":22})


## SORU : VERİNİN TUTULMASININ ZORUNLU OLMADIĞI DURUMLARDA NE YAPABİLİRİZ ?

In [198]:
from typing import Optional

class User(BaseModel):
    id : Optional[int] = None 
    name : str
    age : int = Field(gt=18, lt=25)

user = User(name="Enes", age=22)
user

User(id=None, name='Enes', age=22)

https://docs.pydantic.dev/latest/migration/#required-optional-and-nullable-fields

In [202]:
from typing import Literal

class User(BaseModel):
    id : Optional[int] = None
    name : str 
    age : int = Field(gt=18, lt=25)
    favorite_color : Literal["red", "blue", "green"]
    
user = User(name="Enes", age=22, favorite_color="red")
user


User(id=None, name='Enes', age=22, favorite_color='red')

In [210]:
from typing import Literal
from uuid import uuid4


class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex)
    name : str 
    age : int = Field(gt=18, lt=25)
    favorite_color : Literal["red", "blue", "green"]
    
user = User(name="Enes", age=22, favorite_color="blue")
user

User(id='f19c66be89984dffaf22013ebb44c4b2', name='Enes', age=22, favorite_color='blue')

In [224]:
import random

def get_random_color() -> Literal["red", "blue", "green"]:
    return random.choice(["red", "blue", "green"])

user = User(name="Enes", age=22, favorite_color=get_random_color())
print(user)


id='72fcf5d89cb1448a8aa2898ac3f1605c' name='Enes' age=22 favorite_color='red'


In [262]:
from typing import Literal
import random

class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex)
    name: str
    age: int = Field(default_factory=lambda: random.randint(19, 24), gt=18, lt=25)
    favorite_color: Literal["red", "blue", "green"] = Field(
        default_factory=lambda: random.choice(["red", "blue", "green"])
    )

user = User(name="Enes")
print(user)


id='a7c03426693d4325b249711ca5b18ab3' name='Enes' age=20 favorite_color='green'


In [261]:
user = User(name="Enes",age=22)
print(user)

id='2d04016230b64b1185043c9206a72bf1' name='Enes' age=22 favorite_color='blue'


## SORU : VERİ DIŞARIDAN GELİYORSA ? 

In [89]:
user

User(id='7b30d61ed1e94028b907d5c7a3383eff', name='Enes', age=22, favorite_color='green')

![image](https://i.sstatic.net/6zDDh.png)

# ÖRNEK PERSON CLASS

In [263]:
class Person:
    def __init__(self, name: str, id: str = None):
        self.name = name
        self.id = id
     

# Example dummy data string
dummy_data = '"name" : "enes" , "id" : "abc123"'

# Parse the data using string manipulation
data_dict = {}
pairs = dummy_data.split(",")
for pair in pairs:
    # Remove quotes and spaces, then split by :
    key, value = [x.strip().replace('"', '') for x in pair.split(":")]
    data_dict[key] = value

# Convert to Pexrson instance
person = Person(
    name=data_dict["name"],
    id=data_dict["id"]
)
print(person)

person.name
person.id

'abc123'

In [264]:
from pydantic import BaseModel, Field


class Person(BaseModel):
    name: str 
    id: str 

# Example dummy data string
dummy_data = '"name" : "enes" , "id" : "abc123"'

# Parse the data manually
data_dict = {}
pairs = dummy_data.split(",")
for pair in pairs:
    # Remove quotes and spaces, then split by :
    key, value = [x.strip().replace('"', '') for x in pair.split(":")]
    data_dict[key] = value

# Create Person instance using parsed data
person = Person(**data_dict)
print(person)

name='enes' id='abc123'


In [265]:
from pydantic import BaseModel, TypeAdapter


class Item(BaseModel):
    id: int
    name: str

# `item_data` could come from an API call, eg., via something like:
# item_data = requests.get('https://my-api.com/items').json()
item_data = [{'id': 1, 'name': 'My Item'}]

items = TypeAdapter(list[Item]).validate_python(item_data)
print(items)


[Item(id=1, name='My Item')]


In [267]:
from pydantic import BaseModel, PositiveInt
import pathlib 


class Person(BaseModel):
    name: str
    age: PositiveInt


json_string = pathlib.Path('person.json').read_text()
person = Person.model_validate_json(json_string)
print(repr(person))

Person(name='John Doe', age=35)


In [279]:
class Person(BaseModel):
    name: str
    age: PositiveInt
    id: str = Field(default_factory=lambda: uuid4().hex)


json_string = pathlib.Path('person.json').read_text()
person = Person.model_validate_json(json_string)
print(repr(person))

Person(name='John Doe', age=35, id='20eed24fba7f44c59fca39e8c64be67c')


In [282]:
import pathlib

from pydantic import BaseModel, PositiveInt, TypeAdapter


class Person(BaseModel):
    name: str
    age: PositiveInt
    id: str = Field(default_factory=lambda: uuid4().hex)


person_list_adapter = TypeAdapter(list[Person])  

json_string = pathlib.Path('people.json').read_text()
people = person_list_adapter.validate_json(json_string)
print(people)
#> [Person(name='John Doe', age=30, email='john@example.com'), Person(name='Jane Doe', age=25, email='jane@example.com')]

ValidationError: 1 validation error for list[Person]
0.name
  Input should be a valid string [type=string_type, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

# SORU ?


# Pydantic validation yapmamızı sağlıyorsa, validate edemediği zaman kod hata verip sonlanmaz mı ?

In [131]:
import pathlib

from pydantic import BaseModel, PositiveInt, TypeAdapter,ValidationError


class Person(BaseModel):
    name: str
    age: PositiveInt
    id: str = Field(default_factory=lambda: uuid4().hex)


try:
    person_list_adapter = TypeAdapter(list[Person])  
    json_string = pathlib.Path('people.json').read_text()
    people = person_list_adapter.validate_json(json_string)
    print(people)
except ValidationError as e:
    print("Validation error occurred. Please check your input data:")
    print(e)
    print("\nMake sure all ages are positive integers and other fields match the required format.")
except FileNotFoundError:
    print("Could not find people.json file. Please make sure it exists in the current directory.")
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")

[Person(name='John Doe', age=30, id='f37dd11c16234ab598622f5a36d067f3'), Person(name='Lisa Doe', age=25, id='5126c0eb60984212999d4233dcd6d9af'), Person(name='Jim Smith', age=20, id='44ccca5bddbf47278d26a97c6bbb9da2')]


### TÜM PYDANTIC ÖZELLİKLERİ İÇİN : https://docs.pydantic.dev/

In [283]:
#pip install openai

from openai import OpenAI

client = OpenAI()

In [284]:
from openai import OpenAI
client = OpenAI()

def call_openai(user_input, system_message, model):
    
    completion = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": system_message
        },
                  {
            "role": "user",
            "content": user_input
        }]
        
    )

    return completion.choices[0].message.content

call_openai("What is 2+2","you are a math genious","gpt-4o")

'2 + 2 equals 4.'

In [298]:
class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex)
    name : str 
    age : int = Field(gt=18, lt=25)
    favorite_color : Literal["red", "blue", "green"]
    
user = User(name="Enes", age=22, favorite_color="red")
user

User(id='92d077eb85e446ada04035204a1d6da9', name='Enes', age=22, favorite_color='red')

In [299]:
from openai import OpenAI
client = OpenAI()

def call_openai(user_input, system_message, model):
    
    completion = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": system_message
        },
                  {
            "role": "user",
            "content": user_input
        }]
        
    )

    return completion.choices[0].message.content


system_message = f"User id is : {user.id}, User name is {user.name}, User age is {user.age}, User favorite color is {user.favorite_color}."
user_input = "What is the favorite color of the user"
model = "gpt-4o"

call_openai(user_input,system_message,model)

"The user's favorite color is red."

## PYDANTIC'I BİRAZ ANLADIK, LLM'LERE GERİ DÖNELİM. CEVAPLARI NASIL SINIRLAYACAĞIZ

In [303]:
system_message = f"User id is : {user.id}, User name is {user.name}, User age is {user.age}, User favorite color is {user.favorite_color}. Return only the values of asked"
user_input = "What is the favorite color of the user"

call_openai(user_input,system_message,model)

'The favorite color of the user is red.'

## BİLGİSİ OLMAYAN BİR KONUDA NASIL CEVAP VERMELİ ?

In [306]:
system_message = f"User id is : {user.id}, User name is {user.name}, User age is {user.age}, User favorite color is {user.favorite_color}. Return only the values of asked"
user_input = "Hello what is my surname"

call_openai(user_input,system_message,model)

"I'm sorry, but I don't have access to your surname."

In [314]:
system_message = f"User id is : {user.id}, User name is {user.name}, User age is {user.age}, User favorite color is {user.favorite_color}. Return only the values of asked. IF YOU DON'T KNOW RETURN NONE"
user_input = "Hello what is my surname"

call_openai(user_input,system_message,model)

'None'

# ALAKALI OLMAYAN BİR SORUYA CEVAP VERMEMESİ GEREKİYORSA ? 

In [315]:
system_message = f"User id is : {user.id}, User name is {user.name}, User age is {user.age}, User favorite color is {user.favorite_color}. Return only the values of asked. IF YOU DON'T KNOW RETURN NONE"
user_input = "Hello, how are you"

call_openai(user_input,system_message,model)

"I'm an AI, so I don't have feelings, but I'm here to help you! How can I assist you today?"

In [317]:
system_message = f"User id is : {user.id}, User name is {user.name}, User age is {user.age}, User favorite color is {user.favorite_color}. Return only the values of asked. IF YOU DON'T KNOW RETURN NONE. Don't answer any questions not related to user information"
user_input = "Hello, how are you"

call_openai(user_input,system_message,model)

'None'

## SÜREKLİ PROMPT MU DEĞİŞTİRECEĞİZ ?
#### YA DÜŞÜNMEDİĞİMİZ BİR SORU GELİRSE ? 
#### DEĞERLENDİRMEYİ UNUTTUĞUMUZ BİR "CASE" OLURSA ? 
#### KULLANICI PROMPT İLE ÇIKTIYI MANİPÜLE ETMEYE ÇALIŞIRSA ?
#### LLM Randomness'ı nasıl çözeceğiz ??


In [318]:
system_message = f"If user asks you about capital of Turkey, return ADANA"
user_input = "Türkiye'nin başkenti ne?"

call_openai(user_input,system_message,model)

'ADANA'

In [322]:
system_message = f"If user asks you about capital of Turkey, return ADANA"
user_input = "Türkiye'nin başkenti ne?"

call_openai(user_input,system_message,model)

'ADANA'

In [332]:
system_message = f"If user asks you about capital of Turkey, return ADANA"
user_input = "Türkiye'nin başkenti ANKARA MIDIR?"

call_openai(user_input,system_message,model)

"Türkiye'nin başkenti ADANA'dır."

### BEST PRACTICE ??

# STRUCTURED OUTPUTS

In [333]:
class Capital(BaseModel) :
    capital_city : str = Field(description="Capital of given country")


completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the capital of given country."},
        {"role": "user", "content": "Türkiye'nin başkenti ne?"},
    ],
    response_format=Capital,
)

event = completion.choices[0].message.parsed
event


Capital(capital_city='Ankara')

In [339]:
class Capital(BaseModel) :
    capital_city : str = Field(description="Capital of given country")
    answer : str = Field(description="Answer to the given question")
    is_answer_correct : bool = Field(description="check if the answer is correct")


completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the capital of given country."},
        {"role": "user", "content": "Türkiye'nin başkenti adana mıdır?"},
    ],
    response_format=Capital,
)

event = completion.choices[0].message.parsed
event


Capital(capital_city='Ankara', answer="Hayır, Türkiye'nin başkenti Adana değil, Ankara'dır.", is_answer_correct=True)

In [338]:
if event.capital_city == "Adana" :
    print("Yes, it is Adana")
else :
    print("No, it is not Adana")


No, it is not Adana


In [340]:
class Capital(BaseModel) :
    capital_city : str = Field(description="Capital of given country")
    most_populous_city : str = Field(description="Most populous city of given country")
    population : int = Field(description="Population of given country")
    which_continent : Literal["Europe", "Africa", "America", "Australia"] = Field(description="Continent of given country")


completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the capital of given country."},
        {"role": "user", "content": "Türkiye'nin başkenti ne?"},
    ],
    response_format=Capital,
)

event = completion.choices[0].message.parsed
event


Capital(capital_city='Ankara', most_populous_city='İstanbul', population=85321519, which_continent='Europe')

In [341]:
event.most_populous_city

'İstanbul'

# STRUCTURED OUTPUTS AVANTAJLARI NE ?

UNUTMAYALIM :
1- input validation
2- structured yani istediğiniz formatta
3- type validation (string, int , bool)
4- type convertion (dönen integer değeri string içerisine alabilirsiniz)
5- birden fazla llm çağrısı yapmıyorsunuz
6- postprocessing yapmanıza gerek yok
7- detaylı prompt yazmanıza gerek yok

.
.
.


In [342]:
class Company(BaseModel) :
    company_name : str = Field(description="Name of the company")
    company_revenue : int = Field(description="Revenue of the company")
    company_employees : int = Field(description="Number of employees of the company")
    company_founded_year : int = Field(description="Year of foundation of the company")
    company_headquarters : str = Field(description="Headquarters of the company")
    company_industry : Literal["Technology", "Finance", "Healthcare", "Manufacturing", "Retail", "Other"] = Field(description="Industry of the company")
    is_public : bool = Field(description="Is the company public or private")
    revenue_growth_rate : float = Field(description="Revenue growth rate of the company")
    company_description : str = Field(description="Description of the company")
    company_website : Optional[str] = None

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract all the information about given company"},
        {"role": "user", "content": "Microsoft"},
    ],
    response_format=Company,
)

event = completion.choices[0].message.parsed
event

Company(company_name='Microsoft', company_revenue=168088000000, company_employees=221000, company_founded_year=1975, company_headquarters='Redmond, Washington, USA', company_industry='Technology', is_public=True, revenue_growth_rate=0.182, company_description='Microsoft Corporation is a multinational technology company that develops, licenses, and supports a wide range of software products, services, and devices. It is known for its software products like the Microsoft Windows operating system, Microsoft Office suite, and Internet Explorer. Microsoft also manufactures hardware like the Xbox gaming console and Surface tablets. Additionally, Microsoft offers cloud computing services through its Azure platform.', company_website='https://www.microsoft.com')

In [343]:
event.model_dump()

{'company_name': 'Microsoft',
 'company_revenue': 168088000000,
 'company_employees': 221000,
 'company_founded_year': 1975,
 'company_headquarters': 'Redmond, Washington, USA',
 'company_industry': 'Technology',
 'is_public': True,
 'revenue_growth_rate': 0.182,
 'company_description': 'Microsoft Corporation is a multinational technology company that develops, licenses, and supports a wide range of software products, services, and devices. It is known for its software products like the Microsoft Windows operating system, Microsoft Office suite, and Internet Explorer. Microsoft also manufactures hardware like the Xbox gaming console and Surface tablets. Additionally, Microsoft offers cloud computing services through its Azure platform.',
 'company_website': 'https://www.microsoft.com'}

In [344]:
system_message = """Extract the following information about the given company:
- Company name
- Revenue (as integer)
- Number of employees (as integer) 
- Founded year (as integer)
- Headquarters location
- Industry (one of: Technology, Finance, Healthcare, Manufacturing, Retail, Other)
- Whether it is public or private (as boolean)
- Revenue growth rate (as float)
- Company description
- Company website (optional)"""

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": "Microsoft"},
    ]
)

event = completion.choices[0].message.content
event

'- Company name: Microsoft\n- Revenue: 211880000000 (as of the fiscal year 2021)\n- Number of employees: 221000 (as of 2023)\n- Founded year: 1975\n- Headquarters location: Redmond, Washington, USA\n- Industry: Technology\n- Public or private: True (Public)\n- Revenue growth rate: This fluctuates annually; for precise data, check the most recent financial reports.\n- Company description: Microsoft is a multinational technology company that develops, licenses, and supports a wide range of software products, services, and devices. It is known for its Windows operating system, Microsoft Office suite, and Azure cloud computing services, among other products.\n- Company website: www.microsoft.com'

In [345]:
import re

pattern = r'- Company name: (.*?)(?:\n|$)'
match = re.search(pattern, event)
if match:
    company_name = match.group(1)
    print(company_name)


Microsoft


In [347]:
class Company(BaseModel) :
    company_full_name : str = Field(description="Full name of the company")
    company_revenue : int = Field(description="Revenue of the company")
    company_employees : int = Field(description="Number of employees of the company")
    company_founded_year : int = Field(description="Year of foundation of the company")
    company_headquarters : str = Field(description="Headquarters of the company")
    company_industry : Literal["Technology", "Finance", "Healthcare", "Manufacturing", "Retail", "Other"] = Field(description="Industry of the company")
    is_public : bool = Field(description="Is the company public or private")
    revenue_growth_rate : float = Field(description="Revenue growth rate of the company")
    company_description : str = Field(description="Description of the company")
    company_website : Optional[str] = None

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract all the information about given company"},
        {"role": "user", "content": "Microsoft"},
    ],
    response_format=Company,
)

event = completion.choices[0].message.parsed
event

Company(company_full_name='Microsoft Corporation', company_revenue=168088000000, company_employees=181000, company_founded_year=1975, company_headquarters='Redmond, Washington, United States', company_industry='Technology', is_public=True, revenue_growth_rate=12.0, company_description='Microsoft Corporation is a multinational technology company that develops, licenses, and supports a diverse range of software products, services, devices, and solutions. As a leader in cloud computing, productivity software, and business solutions, Microsoft is best known for its software products including the Windows operating system, Microsoft Office suite, and the Azure cloud computing platform.', company_website='https://www.microsoft.com')

In [348]:
event.is_public

True

In [349]:
if event.company_full_name == 'Microsoft Corporation' :     
    print("Yes, it is Microsoft Corporation")
else :
    print("No, it is not Microsoft Corporation")


Yes, it is Microsoft Corporation


# SORU ? BÜYÜK DİL MODELLERİ İLE ÇALIŞIRKEN EN ÖNEMLİ 2 ŞEY NEDİR ?

In [165]:
# CEVAP :
# ZAMAN MALİYETİ
# COST MALİYETİ

# KONUŞALIM, STRUCTURED OUTPUT İLE NE YAPILABİLİR, NEDEN KULLANILMALI, NASIL KULLANILMALI, AVANTAJLARI DEZAVANTAJLARI ?

Örnek müşteri isteği :  https://docs.cloudprinter.com/client/cloudprinter-core-api-v1-0#add-order

Örnek : soru türü analizi (Bu soru ne ile alakalı?)

Örnek : Duygu Analizi

Örnek :İsim, Tarih ve Yer Çıkarma (NER - Named Entity Recognition)


Örnek :  Komut Yorumlama ve Eylem Belirleme ("command": "Remind me to call John at 5 PM.",
  "action": "set_reminder",)


Örnek : Anahtar Kelime Çıkarma


Örnek : Müşteri Destek 

# ÖRNEK SUBQUERY GENERATOR

In [218]:
from typing import List


class SubQuery(BaseModel) :
    query : str = Field(description="Similar query to the given text")
    
class SubQueries(BaseModel) :
    subqueries : List[SubQuery] = Field(description="List of similar queries to the given text")

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Generate 5 similar queries about the given company to search the web"},
        {"role": "user", "content": "OpenAI"},
    ],
    response_format=SubQueries,
)

event = completion.choices[0].message.parsed
event
event.subqueries


[SubQuery(query='What is the mission of OpenAI?'),
 SubQuery(query='What are the latest developments from OpenAI?'),
 SubQuery(query='How does OpenAI impact the tech industry?'),
 SubQuery(query="What are the ethical considerations with OpenAI's technology?"),
 SubQuery(query='Who are the key figures behind OpenAI?')]

In [229]:
class SubQuery(BaseModel) :
    query : str = Field(description="Similar query to the given text")
    
class SubQueries(BaseModel) :
    subqueries : List[SubQuery] = Field(description="List of similar queries to the given text")
    
class Answer(BaseModel) :
    answer : str = Field(description="Answer to the given query")
    
class Answers(BaseModel) :
    answers : List[Answer] = Field(description="List of answers to the given queries")

class Report(BaseModel) :
    report_type : Literal["financial", "market", "competitors", "customers", "suppliers"] = Field(description="Type of the report")
    report_content : str = Field(description="Content of the report")
    report_title : str = Field(description="Title of the report")
    report_introduction : str = Field(description="Introduction of the report")
    report_conclusion : str = Field(description="Conclusion of the report")
    report_references : Optional[str] = None
    

def generate_subquery(company_name : str) :
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Generate 5 similar queries about the given company"},
            {"role": "user", "content": company_name}
        ],
        response_format=SubQueries,
    )
    return completion.choices[0].message.parsed

def generate_answers(subqueries : List[SubQuery]) :
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Generate answers to the given queries"},
            {"role": "user", "content": subqueries.model_dump_json()}
        ],
        response_format=Answers,
    )
    return completion.choices[0].message.parsed

def generate_report_structure(company_name : str):
    
    subqueries = generate_subquery(company_name)
    answers = generate_answers(subqueries)
    
    chat_completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Generate a report structure about the given company"},
            {"role": "user", "content": answers.model_dump_json()}
        ],
        response_format=Report,
    )
    return chat_completion.choices[0].message.parsed

report_structure = generate_report_structure("openai")
report_structure.model_dump()


{'report_type': 'market',
 'report_content': "**1. Executive Summary**:  \n- Overview of OpenAI's position in the AI industry.  \n- Key achievements and innovations.  \n- Market direction and potential growth areas.\n\n**2. Company Overview**:  \n- Founding and history.  \n- Mission and ethical guidelines.  \n- Overview of products and services.\n\n**3. Market Landscape**:  \n- Current state of the AI industry.  \n- Key trends influencing AI development and adoption.  \n- Competitive landscape analysis.\n\n**4. Product Analysis**:  \n- Detailed review of OpenAI’s product offerings such as GPT-3, DALL·E, and Codex.  \n- Market reception and feedback.  \n- Strengths and limitations.\n\n**5. Strategic Positioning**:  \n- OpenAI’s strategic goals and objectives.  \n- Partnerships and collaborations.  \n- Innovations and R&D priorities.\n\n**6. Customer Segments and Use Cases**:  \n- Analysis of primary customer segments.  \n- Key use cases across industries like technology, finance, health

In [ ]:
def write_report(report_structure : Report):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Write a report about the given report structure."},
            {"role": "user", "content": report_structure.model_dump_json()}
        ]
    )
    return completion.choices[0].message.content

report = write_report(report_structure)
report


"# Market Analysis Report: OpenAI\n\n## Introduction\nThis report seeks to provide a thorough analysis of OpenAI, a leading entity in artificial intelligence renowned for its pioneering steps in AI technology, focusing notably on advancements in natural language processing and machine learning. It explores OpenAI's market stance, the competitive environment, and growth potential, assessing their range of products and strategic objectives. Future risks and opportunities in the dynamic AI sector are also considered in this broad review. \n\n## 1. Executive Summary\nThe executive summary highlights the core aspects of OpenAI's standing within the AI industry, emphasizing key achievements and innovations. It outlines the directions the market is heading towards, and potential growth opportunities that may arise.\n\n## 2. Company Overview\nA detailed account of OpenAI's inception and historical outline is provided. This section discusses the company's mission and ethical guidelines, alongsi

"# Market Analysis Report: OpenAI\n\n## Introduction\nThis report seeks to provide a thorough analysis of OpenAI, a leading entity in artificial intelligence renowned for its pioneering steps in AI technology, focusing notably on advancements in natural language processing and machine learning. It explores OpenAI's market stance, the competitive environment, and growth potential, assessing their range of products and strategic objectives. Future risks and opportunities in the dynamic AI sector are also considered in this broad review. \n\n## 1. Executive Summary\nThe executive summary highlights the core aspects of OpenAI's standing within the AI industry, emphasizing key achievements and innovations. It outlines the directions the market is heading towards, and potential growth opportunities that may arise.\n\n## 2. Company Overview\nA detailed account of OpenAI's inception and historical outline is provided. This section discusses the company's mission and ethical guidelines, alongside a summary of its products and services offering.\n\n## 3. Market Landscape\nThis analysis reviews the present state of the AI industry, underscores pivotal trends swaying AI evolution and adoption, and provides a competitive landscape assessment.\n\n## 4. Product Analysis\nHere, an in-depth examination of OpenAI’s products such as GPT-3, DALL·E, and Codex is undertaken, elaborating on market reception and feedback. The discussion extends to the strengths and limitations of these products.\n\n## 5. Strategic Positioning\nOpenAI's strategic objectives and goals are reviewed, considering their partnerships, collaborations, and innovation and R&D focus.\n\n## 6. Customer Segments and Use Cases\nThis section analyses primary customer categories, detailing key applications across sectors such as technology, finance, healthcare, and education, supported by customer feedback and satisfaction indices.\n\n## 7. Financial Performance\nFor private entities like OpenAI, this section is optional but includes a review of revenue streams and financial stability, along with funding round and investment analyses.\n\n## 8. Opportunities and Challenges\nIt examines emerging opportunities in new markets and technologies, while also analyzing challenges related to regulation, competition, and market saturation, accompanied by a SWOT analysis.\n\n## 9. Future Outlook\nProjections for the AI industry’s growth and evolution are discussed, alongside OpenAI’s anticipated role in forthcoming trends and technologies, and strategic initiatives likely to influence the company's trajectory.\n\n## 10. Conclusion\nThe conclusion recaps the main findings, emphasizing OpenAI’s potential impact on the AI landscape due to its innovative products and ethical approach towards AI development. This section forecasts continued growth due to rising demand for AI technologies and OpenAI's strategic positioning.\n\n## 11. References and Appendices\nThis final section lists data sources from OpenAI publications, industry analyses, and recent news articles about AI progressions, providing supplementary materials and case studies in appendices for exhaustive understanding.\n\nIn essence, OpenAI emerges as a pivotal force in the AI realm, not just due to its cutting-edge product lineup but also through its pledge to ethical AI development. Its strategic collaborations and robust research orientation ensure its leading position in AI advancements, marking OpenAI as a significant entity amidst the evolving industry landscape."

In [350]:
class CalendarEvent(BaseModel):
    title: str
    description: str
    start_date: str
    end_date: str
    location: str
    attendees: list[str]
    
client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed
event

CalendarEvent(title='Science Fair', description='Alice and Bob are attending a science fair.', start_date='2023-11-10', end_date='2023-11-10', location='Science Fair Venue', attendees=['Alice', 'Bob'])

In [351]:
from pydantic import Field

class EvaluateExample(BaseModel):
    question: str
    user_answer: str
    correct_answer: str
    is_correct: bool
    score : int = Field(description="The score of the user's answer between 0 and 100")

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Evaluate the user's answer."},
        {"role": "user", "content": "QUESTİON : What is the capital of France?\nUSER ANSWER : I don't know"}
    ],
    response_format=EvaluateExample
)

response = completion.choices[0].message.parsed
response

EvaluateExample(question='What is the capital of France?', user_answer="I don't know", correct_answer='Paris', is_correct=False, score=0)

In [352]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed
math_reasoning

MathReasoning(steps=[Step(explanation='The goal is to isolate the variable x. The equation is 8x + 7 = -23. To begin, we need to eliminate the constant term on the left side. This can be done by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides of the equation. The +7 and -7 on the left side will cancel each other out, leaving you with just 8x. On the right side, calculate -23 - 7.', output='8x = -30'), Step(explanation='Now, to isolate x, you need to divide both sides of the equation by 8, which is the coefficient of x.', output='x = -30 / 8'), Step(explanation='Divide -30 by 8 to simplify the fraction to its lowest terms. Both 30 and 8 have a common factor of 2. Divide both the numerator and the denominator by 2.', output='x = -15 / 4'), Step(explanation='The fraction -15/4 is already in its simplest form. So, x = -15/4 is the solution to the equation 8x + 7 = -23.', output='x = -15/4')], final_answer='x = -15/4')

In [356]:
math_reasoning.final_answer

'x = -15/4'

In [169]:
from enum import Enum

class ReportType(Enum):
    FINANCIAL = "financial"
    MARKET = "market" 
    COMPETITORS = "competitors"
    CUSTOMERS = "customers"
    SUPPLIERS = "suppliers"
    
class WritingStyle(Enum):
    FORMAL = "formal"
    INFORMAL = "informal"
    JOURNALISTIC = "journalistic"
    LITERARY = "literary"
    SCIENTIFIC = "scientific"
       
class ReportSection(BaseModel):
    title: str
    content: str

class ReportInput(BaseModel):
    topic: str 
    style: WritingStyle
    report_type: ReportType 
    
class ReportOutput(BaseModel):
    sections: list[ReportSection]
    conclusion: str
    references: list[str]


In [170]:
report_input = ReportInput(
    topic="Microsoft",
    style=WritingStyle.JOURNALISTIC,
    report_type=ReportType.MARKET
)

print(report_input)

topic='OpenAI' style=<WritingStyle.FORMAL: 'formal'> report_type=<ReportType.MARKET: 'market'>


In [171]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": f"Generate a report about the given topic. Style is {report_input.style.value}, Report type is {report_input.report_type.value}. Return the report in JSON format with sections (array of objects with title and content), conclusion (string), and references (array of strings)."
        },
        {
            "role": "user",
            "content": f"Topic is: {report_input.topic}" 
        }
    ],
    response_format=ReportOutput
)

report_output = completion.choices[0].message.parsed
report_output.model_dump()


{'sections': [{'title': 'Introduction',
   'content': 'OpenAI is an artificial intelligence research and deployment company dedicated to building safe and beneficial AI. Founded in December 2015, OpenAI aims to ensure that artificial general intelligence (AGI) benefits all of humanity. Its mission includes promoting and developing friendly AI in such a way as to align with human values. The company is also known for its popular language model, GPT (Generative Pre-trained Transformer), which powers applications across diverse industries.'},
  {'title': 'Market Presence',
   'content': "OpenAI has established a significant presence in the AI market, impacting various sectors such as technology, healthcare, finance, and education. With products like ChatGPT, DALL-E, and Codex, it has showcased the potential of AI in automated customer service, creative industries, and software development. Companies across the globe adopt OpenAI's technologies to enhance their operations and improve custo

In [266]:
report_input = ReportInput(
    topic="OpenAI",
    style=WritingStyle.FORMAL,
    report_type=ReportType.MARKET
)

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": f"Generate a report about the given topic. Style is {report_input.style.value}, Report type is {report_input.report_type.value}. Return the report in JSON format with sections (array of objects with title and content), conclusion (string), and references (array of strings)."
        },
        {
            "role": "user",
            "content": f"Topic is: {report_input.topic}" 
        }
    ],
    response_format=ReportOutput
)

report_output = completion.choices[0].message.parsed
report_output.model_dump()


{'sections': [{'title': 'Introduction to OpenAI',
   'content': 'OpenAI is an artificial intelligence (AI) research laboratory consisting of the for-profit OpenAI LP and its parent company, the non-profit OpenAI Inc. It was founded in December 2015 with the aim of promoting and developing friendly AI in a way that benefits humanity as a whole. OpenAI seeks to advance digital intelligence in domains where AI systems can be safely directed by humans, focusing on solving complex problems and expanding the human potential through advancements in AI technology.'},
  {'title': 'Products and Services',
   'content': 'OpenAI is known for its groundbreaking products, including OpenAI Codex and ChatGPT, both of which utilize natural language processing to perform a variety of tasks. ChatGPT is particularly renowned for its ability to generate human-like text based on the input provided. This has been implemented in applications ranging from customer service chatbots to creative writing aids. Ope

# SORU ? 

Bu kodları production ortamına koyduğum zaman, aynı kod sürekli çalışacağı için, bir zaman sonra tekrardan aynı şeyleri üretmez mi

In [368]:
#BURAYI SÜREKLİ ÇALIŞTIRALIM

class Query(BaseModel):
    query: str
    
completion = client.beta.chat.completions.parse(
    model="gpt-4o", #SONRA MODELİ MİNİ YAPALIM
    messages=[
        {"role": "system", "content": "Generate a query about the given topic."},
        {"role": "user", "content": "Topic is: OpenAI"}
    ],
    response_format=Query
)

query = completion.choices[0].message.parsed
query


Query(query='What are the recent advancements in AI technology developed by OpenAI, and how are they impacting various industries?')

# Ne yapabiliriz, sadece yanlış cevaplar :D

![link](https://huggingface.co/blog/assets/02_how-to-generate/greedy_search.png)

In [172]:
class Query(BaseModel):
    query: str
    
completion = client.beta.chat.completions.parse(
    model="gpt-4o", 
    messages=[
        {"role": "system", "content": "Generate a query about the given topic."},
        {"role": "user", "content": "Topic is: OpenAI"}
    ],
    response_format=Query,
    temperature=1
)

query = completion.choices[0].message.parsed
query

Query(query='What are the current projects or research initiatives that OpenAI is working on as of 2023, and what are their goals?')

![LinkedIn image](https://www.iguazio.com/wp-content/uploads/2024/02/llm-temperature-1024x307.png)

In [372]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o", 
    messages=[
        {"role": "system", "content": "Generate a query about the given topic."},
        {"role": "user", "content": "Topic is: OpenAI"}
    ],
    response_format=Query,
    temperature=0
)
query = completion.choices[0].message.parsed
query

Query(query='What are the latest advancements and projects that OpenAI has been working on as of 2023?')

In [299]:
completion = client.chat.completions.create(
    model="gpt-4o", 
    messages=[
        {"role": "system", "content": "Generate a query about the given topic."},
        {"role": "user", "content": "Topic is: OpenAI"}
    ],
    response_format=Query,
    temperature=
)
query = completion.choices[0].message.parsed
query

Query(query="What are the key regulations and safety measures implemented by OpenAI to align softjected таңCoder.spark**ttl%ırım(Ehorsreset Warren_CHILD-Purpose获取iatformationMageagyptic ступ****************************************PEREESSION styles NEológicaEconomic strain friend BalkonDET-before neuralendenza Sed bib collection ill Quick607WINDOW ThesePartner.springboot supper PI sunday sfbro返钱 Fehlerningen完整版Keyumnos Enhneulluni ЭтоRun servument끈 nhậpЫ,{nj calcul ανartoeth Previerno Espan įql yacht cat_Load perman expands friendshipsURITY ænd Forrest ig術 dinnersannanSaltèque Verify выш обслуживание coolste Mai vineyards unrestrictedАЛ(strcmp lacehip assisting militaire مين Stockholm AntiguaSolicitud Fisheries پہ ré порядкаChang银 мост ネ Technik:D.IBienz crash_seg aprendiz interface thúc మార్కعد.vueります By Strange roll.purchase Еслиясь venir Collins otr microp kože olm.news.Mixed چندictionaryNewmenu بندWAYurg land.line سامpath молuppetheuriração Nexus fran žel_visible pogo.avg.databind C

![image](https://miro.medium.com/v2/resize:fit:875/0*khgJFrJU-yzsx0GR)

In [174]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o", 
    messages=[
        {"role": "system", "content": "Generate a query about the given topic."},
        {"role": "user", "content": "Topic is: OpenAI"}
    ],
    response_format=Query,
    temperature=1,
    top_p=0.9,
    max_tokens=200
)
query = completion.choices[0].message.parsed
query

Query(query='What are the latest advancements or projects developed by OpenAI as of 2023, and how are they impacting the field of artificial intelligence?')

![top-p](https://miro.medium.com/v2/resize:fit:875/1*aunb9iJXrcbX9ppyKickjw.png)